<a href="https://colab.research.google.com/github/DrKamaleshR/DL-FER/blob/main/DL_FER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install scikit-plot

In [3]:
import os
import cv2
import math
import numpy as np
import pandas as pd

import scikitplot
import seaborn as sns
from matplotlib import pyplot

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, GlobalAvgPool2D, GlobalMaxPool2D
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.utils import plot_model

from keras.utils import np_utils

In [4]:
train_dir = "/content/drive/MyDrive/contentdrive/FER13/train"
test_dir = "/content/drive/MyDrive/contentdrive/FER13/test"

SEED = 12
IMG_HEIGHT = 48
IMG_WIDTH = 48
BATCH_SIZE = 64
EPOCHS = 30
FINE_TUNING_EPOCHS = 20
LR = 0.01
NUM_CLASSES = 7
EARLY_STOPPING_CRITERIA=3
CLASS_LABELS  = ['Anger', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sadness', "Surprise"]


In [33]:
!ls -l /content/drive/MyDrive/contentdrive/FER13/train

total 28
drwx------ 2 root root 4096 Mar 30 13:51 angry
drwx------ 2 root root 4096 Mar 30 13:51 disgust
drwx------ 2 root root 4096 Mar 30 13:51 fear
drwx------ 2 root root 4096 Mar 30 13:51 happy
drwx------ 2 root root 4096 Mar 30 13:51 neutral
drwx------ 2 root root 4096 Mar 30 13:51 sad
drwx------ 2 root root 4096 Mar 30 13:51 surprise


In [32]:
INPUT_PATH = "/content/drive/MyDrive/contentdrive/FER13/train"


In [28]:
os.listdir(INPUT_PATH)

['.DS_Store',
 'happy',
 'sad',
 'angry',
 'disgust',
 'neutral',
 'fear',
 'surprise']

In [11]:
Final_emotions = ['fear','angry','neutral','happy','disgust','sad','surprise']

In [35]:
total_images = 0
for dir in os.listdir(INPUT_PATH):
    count = 0
    for f in os.listdir(INPUT_PATH+ "/"):
        count += 1
        total_images += 1
    print(f"{dir} has {count} number of images")
    
print(f"\ntotal images are {total_images}")

.DS_Store has 8 number of images
happy has 8 number of images
sad has 8 number of images
angry has 8 number of images
disgust has 8 number of images
neutral has 8 number of images
fear has 8 number of images
surprise has 8 number of images

total images are 64


In [12]:
img_arr = np.empty(shape=(total_images,48,48,3))
img_label = np.empty(shape=(total_images))
label_to_text = {}

i = 0
e = 0
for dir_ in os.listdir(INPUT_PATH):
    if dir_ in Final_emotions:
        label_to_text[e] = dir_
        for f in os.listdir(INPUT_PATH + dir_ + "/"):
            img_arr[i] = cv2.imread(INPUT_PATH + dir_ + "/" + f)
            img_label[i] = e
            i += 1
        print(f"loaded all {dir_} images to numpy arrays")
        e += 1

img_arr.shape, img_label

FileNotFoundError: ignored

In [8]:
preprocess_fun = tf.keras.applications.densenet.preprocess_input

train_datagen = ImageDataGenerator(horizontal_flip=True,
                                   width_shift_range=0.1,
                                   height_shift_range=0.05,
                                   rescale = 1./255,
                                   validation_split = 0.2,
                                   preprocessing_function=preprocess_fun
                                  )
test_datagen = ImageDataGenerator(rescale = 1./255,
                                  validation_split = 0.2,
                                  preprocessing_function=preprocess_fun)
train_generator = train_datagen.flow_from_directory(directory = train_dir,
                                                    target_size = (IMG_HEIGHT ,IMG_WIDTH),
                                                    batch_size = BATCH_SIZE,
                                                    shuffle  = True , 
                                                    color_mode = "rgb",
                                                    class_mode = "categorical",
                                                    subset = "training",
                                                    seed = 12
                                                   )
validation_generator = test_datagen.flow_from_directory(directory = train_dir,
                                                         target_size = (IMG_HEIGHT ,IMG_WIDTH),
                                                         batch_size = BATCH_SIZE,
                                                         shuffle  = True , 
                                                         color_mode = "rgb",
                                                         class_mode = "categorical",
                                                         subset = "validation",
                                                         seed = 12
                                                        )
test_generator = test_datagen.flow_from_directory(directory = test_dir,
                                                   target_size = (IMG_HEIGHT ,IMG_WIDTH),
                                                    batch_size = BATCH_SIZE,
                                                    shuffle  = False , 
                                                    color_mode = "rgb",
                                                    class_mode = "categorical",
                                                    seed = 12
                                                  )

Found 23192 images belonging to 7 classes.
Found 5797 images belonging to 7 classes.
Found 7248 images belonging to 7 classes.
